In [3]:
import sys
from pathlib import Path

DIR = Path('..')
sys.path.append(str(DIR))

import wavetrace as wt

IN_DIR = DIR/'examples'/'inputs'
OUT_DIR = DIR/'examples'/'outputs'

%load_ext autoreload
%autoreload 2

high_definition = False


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# Download elevation data

bounds = [174, -37, 176, -36]
names = wt.get_srtm_tile_names(bounds)
print('Tiles to download =', names)

username = None
password = None
if high_definition:
    path = IN_DIR/'elevations-hd'
    try:
        username = wt.get_secret('NASA_USERNAME')
        password = wt.get_secret('NASA_PASSWORD')
    except:
        pass
else:
    path = IN_DIR/'elevations-sd'
    
wt.download_elevation_data_nasa(bounds, path, high_definition=high_definition,
  username=username, password=password)

Tiles to download = ['S37E174', 'S37E175']


NotImplementedError: Downloading high definition data has not been implemented yet

In [ ]:
# Create SPLAT transmitters data

in_path = IN_DIR/'transmitters.csv'
out_path = OUT_DIR/'splat_files'
wt.create_splat_transmitter_data(in_path, out_path)
% ls {out_path}

In [ ]:
# Create SPLAT elevation data

if high_definition:
    in_path = IN_DIR/'elevations-hd'
else:
    in_path = IN_DIR/'elevations-sd'
out_path = OUT_DIR/'splat_files'

wt.create_splat_elevation_data(in_path, out_path, high_definition=high_definition)
% ls {out_path}

In [ ]:
# Create a coverage maps

transmitter_names = ['MyNetwork_11']
in_path = OUT_DIR/'splat_files'
out_path = OUT_DIR
wt.create_coverage_maps(in_path, out_path, transmitter_names, high_definition=high_definition)
% ls {out_path/transmitter_names[0]}